# Init

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/wliao0504/code/clif/CLIF-MIMIC'

In [ ]:
import numpy as np
import pandas as pd
import logging
import duckdb
import pandera as pa
from importlib import reload
import src.utils as utils
from typing import Dict, List
# reload(utils)
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging, con, REPO_ROOT, mimic_table_pathfinder, \
    search_mimic_items, read_from_rclif, save_to_rclif
from src.utils_qa import all_null_check

loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json


In [3]:
import src.tables.ecmo_mcs as ecmo_mcs
reload(ecmo_mcs)
# test_out = ecmo_mcs._test()
# clif_ecmo = read_from_rclif("ecmo_mcs")

2025-07-29 17:30:02,903 - INFO - initialized logging at logs/etl.log
2025-07-29 17:30:02,905 - INFO - initialized logging at logs/etl.log


<module 'src.tables.ecmo_mcs' from '/Users/wliao0504/code/clif/CLIF-MIMIC/src/tables/ecmo_mcs.py'>

# Dev

In [6]:
query = f"""
SELECT 
    CAST(dx.hadm_id AS VARCHAR) as hospitalization_id,
    CAST(dx.icd_code AS VARCHAR) as diagnosis_code,
    CAST(dx.icd_version AS VARCHAR) as diagnosis_code_format,
    CAST(d.long_title AS VARCHAR) as diagnosis_name,
FROM '{mimic_table_pathfinder('diagnoses_icd')}' dx
LEFT JOIN '{mimic_table_pathfinder('d_icd_diagnoses')}' d
    USING (icd_code, icd_version)
"""
hosp_dx = duckdb.sql(query).df()

In [7]:
save_to_rclif(hosp_dx, 'hospital_diagnosis')

2025-07-29 18:26:48,601 - INFO - saving hospital_diagnosis rclif table as a parquet file at /Users/wliao0504/code/clif/CLIF-MIMIC/src/../output/rclif-dev-test/clif_hospital_diagnosis.parquet.


In [15]:
query = f"""
WITH dx as (
    SELECT *
    FROM hosp_dx
    WHERE diagnosis_code_format = '10' AND diagnosis_code in ('U071', 'U072', 'J1281', 'J1282', 'B9721', 'M3581', 'U099')
)
SELECT diagnosis_code, diagnosis_name, COUNT(*) as n
FROM dx
GROUP BY diagnosis_code, diagnosis_name
"""
covid_dx = duckdb.sql(query).df()
print(covid_dx.shape)
covid_dx.head()

(6, 3)


,diagnosis_code,diagnosis_name,n
0,J1281,Pneumonia due to SARS-associated coronavirus,1
1,B9721,SARS-associated coronavirus as the cause of di...,1
2,U071,COVID-19,3978
3,U099,"Post COVID-19 condition, unspecified",109
4,J1282,Pneumonia due to coronavirus disease 2019,1102


In [17]:
query = f"""
WITH dx AS (
    SELECT hospitalization_id, diagnosis_code
    FROM hosp_dx
    WHERE diagnosis_code IN ('U099', 'U071', 'J1282')
),
u099_only AS (
    SELECT hospitalization_id
    FROM dx
    GROUP BY hospitalization_id
    HAVING COUNT(DISTINCT diagnosis_code) = 1 AND MAX(diagnosis_code) = 'U099'
)
SELECT COUNT(*) as num_hospitalizations
FROM u099_only
"""
u099_only_count = duckdb.sql(query).df()
print(u099_only_count)


   num_hospitalizations
0                   101


# Validate